# Mentoria Data Science Academy

## Imports

In [189]:
import numpy as np
import pandas as pd
import re
import string
import nltk
import os




# Downloads do pacote NLTK
nltk.download('stopwords')
nltk.download('rslp')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joaop\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\joaop\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

## 0.0 Carregando os Dados

In [192]:
data_path = 'Scrapping/dados'
df1 = pd.read_csv(os.path.join(data_path, 'posts.csv'), sep = '|')
df2 = pd.read_csv(os.path.join(data_path, 'posts_2.csv'), sep = '|')

In [196]:
df = pd.concat([df1, df2], axis = 0)

In [197]:
df.head()

,title,date,answers,visits,url,post,area
0,recurso cabível exceção de preexecutividade re...,11 de Março de 2021,1,1.0,threads/recurso-cabivel-excecao-de-preexecutiv...,"\n\t\t\t\t\t\n\t\t\t\t\tBoas tardes, caros col...",direito-do-trabalho.17
1,Liquidez - Reforma Trabalhista,06 de Março de 2021,0,6.0,threads/liquidez-reforma-trabalhista.23092/,"\n\t\t\t\t\t\n\t\t\t\t\tBoa noite, colegas.\n\...",direito-do-trabalho.17
2,Rito Sumário - sentença irrecorrível?,06 de Março de 2021,0,6.0,threads/rito-sumario-sentenca-irrecorrivel.23091/,"\n\t\t\t\t\t\n\t\t\t\t\tOlá, colegas.\n\nAlgué...",direito-do-trabalho.17
3,DISPENSA DE FUNCIONARIO AFASTADO EM ESPECIE DE...,27 de Janeiro de 2021,0,9.0,threads/dispensa-de-funcionario-afastado-em-es...,"\n\t\t\t\t\t\n\t\t\t\t\tPrezados, Drs.\n\nPrim...",direito-do-trabalho.17
4,Cálculo Trabalhista - contador ou advogado?,03 de Maio de 2018,8,1.0,threads/calculo-trabalhista-contador-ou-advoga...,"\n\t\t\t\t\t\n\t\t\t\t\tQueridos (as),\n\nUm c...",direito-do-trabalho.17


## 1.0 Descrição dos dados

### 1.1 Dimensão dos dados

In [198]:
print("Número de exemplos:", df.shape[0])
print("Número de colunas:", df.shape[1])

Número de exemplos: 5826
Número de colunas: 7


### 1.2 Tipo das colunas

In [199]:
# Mostra o tipo de cada variável
df.dtypes

title       object
date        object
answers      int64
visits     float64
url         object
post        object
area        object
dtype: object

### 1.3 Variável target 

### 1.3.1 Classes únicas


In [200]:
# Check de quais classes o dataset possui
df['area'].unique()

array(['direito-do-trabalho.17', 'direito-de-familia.11',
       'direito-administrativo.15',
       'direito-civil-empresarial-e-do-consumidor.10',
       'direito-penal-e-processo-penal.12', 'empresarial'], dtype=object)

In [201]:
print("Número de classes diferentes: ", len(df['area'].unique()))

Número de classes diferentes:  6


### 1.3.2 Verificando o balanceamento das classes

In [202]:
# Verifica em porcentagem o balanço das classes
df['area'].value_counts(normalize=True)*100

direito-civil-empresarial-e-do-consumidor.10    33.882595
direito-de-familia.11                           28.046687
direito-do-trabalho.17                          20.116718
direito-penal-e-processo-penal.12               11.465843
direito-administrativo.15                        6.316512
empresarial                                      0.171644
Name: area, dtype: float64

**Observações:** A classe direito admnistrativo é a que apresenta menos classes.

 ## 2.0 Limpeza dos dados

### 2.1 Visualizando os dados em formato RAW

In [203]:
# Printando as 10 primeiras strings em formato raw
for i, post in enumerate(df['post'][:10]):
    print(repr(df['post'].iloc[i]))
    print('\n')

'\n\t\t\t\t\t\n\t\t\t\t\tBoas tardes, caros colegas\n\nAntes de mais nada, sinto um imenso prazer em visitar esse fórum, por ter me ajudado e muito.\nPreciso da opinião dos colegas, no seguinte assunto trabalhista.\n\nAcontece que a exceção de pre-executividade trabalhista  proposta em favor da cliente fora rejeitada, nesse caso cabe algum recurso....pois segundo uma pesquisa que efetuei, consta que o agravo de petição não é cabível por ser decisão interlocutória, agradeço desde já.\n\t\t\t\t'


'\n\t\t\t\t\t\n\t\t\t\t\tBoa noite, colegas.\n\nCuido de uma ação de cobrança de contribuição sindical rural proposta pelo rito sumário. Por fim o autor requereu: \n\n"a condenação do Réu ao pagamento da quantia atualizada do débito no importe de R$ 835,45 (oitocentos e trinta e cinco reais e quarenta e cinco centavos), referente à Contribuição Sindical Rural do(s) Exercício(s) de 2015, conforme os Demonstrativos da Constituição do Crédito ora inclusos e integrantes desta petição inicial, bem c

## 2.2 Criando um Pipeline de Limpeza

In [204]:
# Essa função substitui mais de um espaço em sequência (\n\t\t\t\t) por apenas um espaço
def remove_seq_breaklines(text):
    clean_text = re.sub('\s{2,}', ' ', text)
    return clean_text

In [87]:
# Essa função substitui \n por um espaço
def remove_breaklines(text):
    clean_text = re.sub('\n', ' ', text)
    return clean_text

In [151]:
# Essa função remove as pontuações e os números (ex: datas serão removidas)
def remove_punctuation(text):
    clean_text  = "".join([char for char in text if char not in string.punctuation])
    clean_text = re.sub('[0-9]+', '', clean_text)
    clean_text = clean_text.replace('º', '')
    return clean_text

In [121]:
# Essa função converte textos em textos lowercase 
def convert_to_lower(text):
    return text.lower()

In [108]:
# Essa função remove as stopwords
def remove_stopwords(text):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    clean_text = [word for word in text.split(' ') if word not in stopwords]
   
    return " ".join(clean_text)


In [179]:
# Essa função aplica o stemming nas palavras
def stemmatization(text):
    stemmer = nltk.stem.RSLPStemmer()
    split_text = text.split(' ')
    stem_text = [stemmer.stem(word) for word in split_text if word != ""]
    stem_text = " ".join(stem_text)
    
    return stem_text

In [206]:
# Função que aplica o pipeline
def run_clean_pipeline(text):
    clean_text = remove_breaklines(text)
    clean_text = remove_seq_breaklines(clean_text)
    clean_text = remove_punctuation(clean_text)
    clean_text = convert_to_lower(clean_text)
    clean_text = remove_stopwords(clean_text)
    clean_text = stemmatization(clean_text)

    return clean_text
    

In [209]:
# Fazendo a limpeza das linhas através do pipeline
clean_posts = list(df['post'].apply(run_clean_pipeline))

In [210]:
# Printa os 10 primeiras posts após a limpeza
for post in clean_posts[:10]:
    print(post)
    print('\n')

boa tard car coleg ant nad sint imens praz visit fórum ter ajud precis opin coleg seguint assunt trabalh acontec exceç preexecut trabalh propost favor client rejeit ness cas cab algum recursopoil segund pesquis efet const agrav pet cab ser decis interlocutór agradeç desd


boa noit coleg cuid açã cobranç contribu sindic rural propost rit sum fim autor requer conden réu pag quant atual débit import r oitocent trint cinc real quarent cinc centav refer contribu sindic rural exercíci conform demonstr constitu crédit ora inclus integr dest pet inic bem réu conden pag jur mult correç monetár art clt dat efet quit cust process honor advocatíci sobr val conden r cent vint cinc real trint centav val estim bas val caus tend vist após advent reform trabalh inic dev ser líqu ent ped liquid moment proposit val jur correç contas apen part dat ajuiz dia efetiv liquid advent nov red artig parágraf part final pod juiz extrapol val indic inic aplic jur correç pretérit ajuiz grat desd


olá coleg alguém 